# Inquinamento Amat:

In [10]:
from datetime import datetime, timedelta
import json
from shapely import Point

import requests
from bs4 import BeautifulSoup, Tag
from tqdm import tqdm
import geopandas as gpd

In [11]:
# RAW
PATH_STAZIONI_RAW = "../Data/Raw/Inquinamento/Amat/inquinamento-amat_stazioni_raw.csv"
PATH_INQUINANTI_RAW = "../Data/Raw/Inquinamento/Amat/inquinamento-amat_inquinanti_raw.csv"
PATH_INGESTION_AMAT_RAW = "../Data/Raw/Inquinamento/Amat/inquinamento-amat_ingestion_raw.json"
PATH_SOGLIE_RAW = "../Data/Raw/Inquinamento/Amat/inquinamento-amat_soglie_raw.csv"

# STAGING
PATH_BOLLETTINO_STAGING = "../Data/Staging/Inquinamento/Amat/inquinamento-amat_bollettino_staging.csv"
PATH_MISURAZIONI_STAGING = "../Data/Staging/Inquinamento/Amat/inquinamento-amat_misurazioni_staging.csv"

# CLEAN
PATH_MISURAZIONI_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_misurazioni_clean.csv"
PATH_BOLLETTINO_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_bollettino_clean.csv"
PATH_STAZIONI_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_stazioni_clean.csv"
PATH_INQUINANTI_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_inquinanti_clean.csv"
PATH_SOGLIE_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_soglie_clean.csv"
PATH_SQLITE_DB_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_sqlite-db_clean.db"

## Scraper

In [15]:
#Scraper functions

def _get_table_row_values(row:Tag) -> dict:
    """
    Return a dict like: {"pollutant": value, (...)}
    """
    inquinanti = [
        "Biossido di Zolfo",
        "Polveri < 10",
        "Polveri < 2.5",
        "Biossido di Azoto",
        "Monossido di carbonio",
        "Ozono",
        "Benzene"
    ]
    cells = row.select("td")[1:]
    row_value = {}
    for pollutant, cell in zip(inquinanti, cells):
        row_value[pollutant] = cell.text
    return row_value

def get_table_day_values(day: datetime) -> dict | None:
    """
    Return a dict like: {"station_address": {"pollutant": value, (...)}, (...)}
    """

    url = f"https://www.amat-mi.it/index.php?id_sezione=35&data_bollettino={day.year}-{day.month}-{day.day}"
    main_page = requests.get(url)
    main_page = BeautifulSoup(main_page.text)
    table = main_page.select_one(".table")
    if not table:
        return None

    table_values = {}
    stations = [
        "Viale Liguria",
        "Viale Marche",
        "Via Pascal",
        "Via Senato",
        "Verziere"
    ]
    rows = table.select("tr")[2:7]
    for station, row in zip(stations, rows):
        table_values[station] = _get_table_row_values(row)
    return table_values

def get_days_list(start: datetime, end: datetime):
    numdays = (end - start).days
    return [start + timedelta(days=x) for x in range(numdays + 1)]

# Other functions

def pad2(day_or_month: int) -> str:
    response = str(day_or_month)
    if len(response) > 1:
        return response
    return f"0{response}"

days = get_days_list(datetime(2023, 1, 1), datetime(2023, 12, 31))
db = []
pbar = tqdm(total=len(days), desc="Scraping progress...")
for day in days:
    table_value = get_table_day_values(day)
    if table_value:
        db.append({"Date": f"{day.year}-{pad2(day.month)}-{pad2(day.day)}", "Stations": table_value})
    pbar.update(1)

Scraping progress...: 100%|██████████| 365/365 [02:12<00:00,  3.30it/s]

In [16]:
with open(PATH_INGESTION_AMAT_RAW, "w") as file:
    json.dump(db, file, indent=4)

### Normalizzazione

In [17]:
# Load raw files to normalizations

with open(PATH_INGESTION_AMAT_RAW, 'r') as file:
    ingestion:list = json.load(file)
stations = pd.read_csv(PATH_STAZIONI_RAW)
inquinanti_df = pd.read_csv(PATH_INQUINANTI_RAW)

# Start operations
bollettino_csv = [["id_bollettino", "data"]]
misurazione_csv = [["id_bollettino", "id_stazione", "id_inquinante", "valore"]]
for i, rilevation in enumerate(ingestion):
    id_bollettino = i + 1
    bollettino_csv.append([id_bollettino, rilevation["Date"]])
    for nome_stazione, inquinanti in rilevation["Stations"].items():
        id_station = stations[stations["nome"] == nome_stazione]["id_stazione"].values[0]
        for nome_inquinante, valore in inquinanti.items():
            id_inquinante = inquinanti_df[inquinanti_df["nome"] == nome_inquinante]["id_inquinante"].values[0]
            misurazione_csv.append([id_bollettino, id_station, id_inquinante, valore])

In [18]:
pd.DataFrame(bollettino_csv).to_csv(PATH_BOLLETTINO_STAGING, index=False, header=False)
pd.DataFrame(misurazione_csv).to_csv(PATH_MISURAZIONI_STAGING, index=False, header=False)

## ELT

### Pulizia tabella misurazione

In [49]:
def lambda_valore(x):
    valore = x.strip("\n").strip("\r").strip().replace(",", ".")
    try:
        valore = float(valore)
    except:
        if "<" in valore:
            valore = 0.0
        else:
            valore = None
    return valore

misurazione = gpd.read_file(PATH_MISURAZIONI_STAGING)

misurazione["valore"] = misurazione["valore"].apply(lambda_valore)
misurazione = misurazione[misurazione["valore"].notna()]

In [51]:
misurazione.to_csv(PATH_MISURAZIONI_CLEAN, index=False)

### Pulizia tabelle e caricamento in Clean

In [53]:
# Nessuna pulizia necessaria al momento. Carico direttamente.

gpd.read_file(PATH_BOLLETTINO_STAGING).to_csv(PATH_BOLLETTINO_CLEAN, encoding='latin',index=False)

gpd.read_file(PATH_STAZIONI_RAW).to_csv(PATH_STAZIONI_CLEAN, encoding='latin', index=False)

gpd.read_file(PATH_INQUINANTI_RAW).to_csv(PATH_INQUINANTI_CLEAN, encoding='latin', index=False)

gpd.read_file(PATH_SOGLIE_RAW).to_csv(PATH_SOGLIE_CLEAN, encoding='latin', index=False)

Creazione Geojson con medie annue

In [8]:
def get_tabella_media_inquinanti(id_inquinante:int, nome_valore:str):
    
    inquinanti = gpd.pd.read_csv(PATH_INQUINANTI_CLEAN, encoding="latin")
    stazioni = gpd.pd.read_csv(PATH_STAZIONI_CLEAN, encoding="latin")
    misurazioni = gpd.pd.read_csv(PATH_MISURAZIONI_CLEAN, encoding="latin")
    misurazioni = misurazioni.rename(columns={"valore": nome_valore})
    bollettino = gpd.pd.read_csv(PATH_BOLLETTINO_CLEAN, encoding="latin")
    bollettino["data"] = gpd.pd.to_datetime(bollettino["data"])

    df = misurazioni[misurazioni["id_inquinante"] == id_inquinante]
    df = df.merge(inquinanti, on="id_inquinante", how="left")
    df = df.merge(bollettino, on="id_bollettino", how="left")
    df = df.merge(stazioni, on="id_stazione", how="left", suffixes=("_pm10", "_stazione"))
    colonne = [nome_valore, "data", "simbolo", "nome_stazione", "latitude", "longitute"]
    df = df[colonne]

    df = df.groupby("nome_stazione").mean("valore").reset_index()
    return df

df_pm10 = get_tabella_media_inquinanti(2, "valore_pm10")
df_pm25 = get_tabella_media_inquinanti(3, "valore_pm25")
df_NO2 = get_tabella_media_inquinanti(4, "valore_NO2")

final_df = df_pm10.merge(df_pm25[["nome_stazione", "valore_pm25"]], on="nome_stazione", how="left" \
                         ).merge(df_NO2[["nome_stazione", "valore_NO2"]], on="nome_stazione", how="left")
final_df = final_df[["valore_pm10", "valore_pm25", "valore_NO2", "nome_stazione", "latitude", "longitute"]]

In [9]:
final_df["geometry"] = final_df.apply(lambda row: Point(row["longitute"], row["latitude"]), axis=1)
PATH_INQUINANTI_AMAT_GEOJSON_CLEAN = "../Data/Clean/Inquinamento/Amat/inquinamento-amat_media_clean.geojson"
gpd.GeoDataFrame(final_df, geometry="geometry", crs="EPSG:6707").to_file(PATH_INQUINANTI_AMAT_GEOJSON_CLEAN, driver="GeoJSON")

## Creation SQLITE DB

In [ ]:
import sqlite3

import pandas as pd

def get_schema_table(drop_command:str, create_command:str, insert_command:str, rows :list[tuple]) -> list[str]:

    for row in rows:
        insert_command = insert_command + "("
        for element in row:
            if isinstance(element, str):
                element = "'" + element + "'"
            insert_command = insert_command + f"{element},"
        insert_command = insert_command.strip(",") + "),"
    insert_command = insert_command.strip(",") + ";"

    return [drop_command, create_command, insert_command]

def execute_query_list(queries: list[str]):
    response = []
    with sqlite3.connect(PATH_SQLITE_DB_CLEAN) as connection:
        for query in queries:
            cursor = connection.cursor()
            result = cursor.execute(query)
            response.append(result.fetchall())
            connection.commit()
    return response

tables = {}
# set commands tables
tables["bollettino"] = {
    "commands":{
        "drop": "DROP TABLE IF EXISTS bollettino;",
        "create": """CREATE TABLE bollettino ("id_bollettino" INT PRIMARY KEY,"data" DATE NULL);""",
        "insert": """INSERT INTO bollettino ("id_bollettino","data") VALUES\n"""
    }
}
tables["stazione"] = {
    "commands": {
        "drop": "DROP TABLE IF EXISTS stazione;",
        "create": """
            CREATE TABLE stazione (
                "id_stazione" INT PRIMARY KEY,
                "nome" VARCHAR(255) NULL,
                "latitude" NUMERIC NULL,
                "longitute" NUMERIC NULL
            );
        """,
        "insert":"""INSERT INTO stazione ("id_stazione","nome","latitude","longitute") VALUES\n"""
    }
}
tables["inquinante"] = {
    "commands": {
        "drop": "DROP TABLE IF EXISTS inquinante;",
        "create": """
            CREATE TABLE inquinante (
                "id_inquinante" INT PRIMARY KEY,
                "nome" VARCHAR(255) NULL,
                "simbolo" VARCHAR(255) NULL,
                "unita_di_misura" VARCHAR(255) NULL,
                "media_temporale" VARCHAR(255) NULL
            );
        """,
        "insert":"""INSERT INTO inquinante ("id_inquinante","nome","simbolo","unita_di_misura","media_temporale") VALUES\n"""
    }
}
tables["misurazione"] = {
    "commands": {
        "drop": """DROP TABLE IF EXISTS misurazione;""",
        "create": """
            CREATE TABLE misurazione (
                "id_bollettino" INT NOT NULL,
                "id_stazione" INT NOT NULL,
                "id_inquinante" INT NOT NULL,
                "valore" NUMERIC NULL,
                FOREIGN KEY(id_bollettino) REFERENCES bollettino(id_bollettino),
                FOREIGN KEY(id_stazione) REFERENCES stazione(id_stazione),
                FOREIGN KEY(id_inquinante) REFERENCES inquinante(id_inquinante)
            );
        """,
        "insert": """INSERT INTO misurazione ("id_bollettino","id_stazione","id_inquinante","valore") VALUES\n"""
    }
}
tables["soglia"] = {
    "commands": {
        "drop": """DROP TABLE IF EXISTS soglia;""",
        "create": """
            CREATE TABLE soglia (
                "id_inquinante" INT NOT NULL,
                "tipo_soglia" VARCHAR(255) NULL,
                "valore" INT NULL,
                "unita_di_misura" VARCHAR(255) NULL,
                "periodo_di_riferimento" VARCHAR(255) NULL,
                "max_superamenti_anno" INT NULL
            );
        """,
        "insert": """INSERT INTO soglia (
                        "id_inquinante",
                        "tipo_soglia",
                        "valore",
                        "unita_di_misura",
                        "periodo_di_riferimento",
                        "max_superamenti_anno"
                    ) VALUES\n"""
    }
}

# set rows from df
tables["bollettino"]["rows"] = list(pd.read_csv(PATH_BOLLETTINO_CLEAN).itertuples(index=False, name=None))
tables["stazione"]["rows"] = list(pd.read_csv(PATH_STAZIONI_CLEAN).itertuples(index=False, name=None))
tables["inquinante"]["rows"] = list(pd.read_csv(PATH_INQUINANTI_CLEAN).itertuples(index=False, name=None))
tables["misurazione"]["rows"] = list(pd.read_csv(PATH_MISURAZIONI_CLEAN).itertuples(index=False, name=None))
tables["soglia"]["rows"] = list(pd.read_csv(PATH_SOGLIE_CLEAN).itertuples(index=False, name=None))

# Create schema query
total_query = []
for v in tables.values():
    total_query.extend(get_schema_table(v["commands"]["drop"], v["commands"]["create"], v["commands"]["insert"], v["rows"]))

# Execute query
query_result = execute_query_list(total_query)

# Inquinamento Airnet

In [53]:
import json
from io import StringIO

import requests
from sseclient import SSEClient
from tqdm import tqdm
import pandas as pd

In [2]:
# RAW
PATH_INQUINAMENTO_INGESTION_RAW = "../Data/Raw/Inquinamento/Airnet/inquinamento-airnet_media_raw.geojson"

# STAGING

# CLEAN
PATH_INQUINAMENTO_INGESTION_CLEAN = "../Data/Clean/Inquinamento/Airnet/inquinamento-airnet_ingestion_clean.geojson"

## Scraping

Otteniamo gli id delle stazioni di Milano

In [3]:
URL = "https://airnet.waqi.info/airnet/validator/check/112273" # Una stazione a caso di milano\
risposta = requests.get(URL)
risposta_json = risposta.json()
neighbors = risposta_json["data"]["neighbors"]  # Da questa riusciamo a ricavare tutte le stazioni "vicine"

inizializziamo il geojson con id e coordinate delle stazioni

In [4]:
geojson = {
    "type": "FeatureCollection",
    "name": "Inquinamento",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::6707"
        }
    },
    "features": []
}
for geo, id in zip(neighbors["geos"], neighbors["ids"]):
    if not id == 353767: # Stazione marco emme, Milan, Italy che non contiene dati pm10 e pm25
        geojson["features"].append(
            {
                "type": "Feature",
                "properties": {
                    "id": id,
                    "longitudine": geo[1],
                    "latitudine": geo[0],
                    "name": "",
                    "pm10": [],
                    "pm25": []
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [geo[1], geo[0]]
                }
            }
        )

Estraiamo i dati di ogni stazione (per i 2 inquinanti) e li inseriamo direttamente tra le "features" >> "properties" del geojson inizializzato

In [5]:
pbar = tqdm(total=len(geojson["features"]), desc="Scraping progress...")

for feature in geojson["features"]:
    for inquinante in ["pm10", "pm25"]:
        url = f"https://airnet.waqi.info/airnet/sse/historic/daily/{feature["properties"]["id"]}?specie={inquinante}"
        response = requests.get(url, params={"specie": inquinante}, headers={"Accept": "text/event-stream"}, stream=True)
        client = SSEClient(response)
        for i, event in enumerate(client.events()):
            if event.event != "error":
                try:
                    data = json.loads(event.data)
                    if i == 0:
                        feature["properties"]["name"] = data["meta"]["name"]
                    elif isinstance(data, dict): #Ogni tanto ci sono righe sporche di stringhe
                        feature["properties"][inquinante].append(data)
                except json.JSONDecodeError:
                    print("Errore JSON:", event.data)
    pbar.update(1)

Scraping progress...: 100%|██████████| 19/19 [01:17<00:00,  4.18s/it]

salvo il file geojson creato in Raw

In [6]:
with open(PATH_INQUINAMENTO_INGESTION_RAW, 'w') as file:
    json.dump(geojson, file, indent=4)

## ETL

A partire dai dati giornalieri dei 2 inquinanti (pm10 e pm25) ricaviamo gli aggregati per anno sottoforma di media delle mediane giornaliere.

In [54]:
def set_mean_pm(row):
    for pm in ["pm10", "pm25"]:
        df = gpd.pd.read_json(StringIO(row[pm]))
        df["day"] = gpd.pd.to_datetime(df["day"])
        df = df[df['day'].dt.year.isin([2023])].reset_index()
        if len(df) > 200:
            row[pm] = df["median"].mean()
        else:
            row[pm] = None
    return row

gdf = gpd.read_file(PATH_INQUINAMENTO_INGESTION_RAW)
gdf = gpd.GeoDataFrame(gdf.apply(set_mean_pm, axis=1), geometry=gdf.geometry.name)

Salviamo su Clean

In [55]:
gdf.to_file(PATH_INQUINAMENTO_INGESTION_CLEAN, driver="GeoJSON")

# Join dei 2 dataframe

In [15]:
import geopandas as gpd

In [16]:
PATH_INQUINAMENTO_JOINED_CLEAN = "../Data/Clean/Inquinamento/inquinamento_media_clean.geojson"